In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import ast
from TwoDimTTC import *
import time

# Step 7

### Load data

In [3]:
# Specify the directory to load files from
directory = r'C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\06_data_with_heading_vectors_folder'
# Get a list of files in the directory
file_name_data_with_pairs_list = os.listdir(directory)
file_name_data_with_pairs_list

['18_data_with_heading_vectors.csv',
 '19_data_with_heading_vectors.csv',
 '20_data_with_heading_vectors.csv',
 '21_data_with_heading_vectors.csv',
 '22_data_with_heading_vectors.csv',
 '23_data_with_heading_vectors.csv',
 '24_data_with_heading_vectors.csv',
 '25_data_with_heading_vectors.csv',
 '26_data_with_heading_vectors.csv',
 '27_data_with_heading_vectors.csv',
 '28_data_with_heading_vectors.csv',
 '29_data_with_heading_vectors.csv']

In [4]:
root_dir = r"C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation"
folder_name='06_data_with_heading_vectors_folder'

# Load dataframes
def load_dfs(file_name_list, root_dir, folder_name):
    os.chdir(root_dir)
    # change to data dir
    os.chdir(folder_name)
    dfs = []
    for i in range(len(file_name_list)):
        df=pd.read_csv(file_name_list[i])
        dfs.append(df)
    # change back to parent directory
    os.chdir('..')
    return dfs

dfs=load_dfs(file_name_data_with_pairs_list, root_dir, folder_name)

In [5]:
# print dfs info
def print_dfs_info(dfs, file_name_data_with_pairs_list):
    for i in range(len(dfs)):
        print(f'{file_name_data_with_pairs_list[i]} df:\n')
        print(dfs[i].info())
        print('------------------------------------------------------\n')

print_dfs_info(dfs, file_name_data_with_pairs_list)

18_data_with_heading_vectors.csv df:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42378 entries, 0 to 42377
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   recordingId    42378 non-null  int64  
 1   trackId        42378 non-null  int64  
 2   frame          42378 non-null  int64  
 3   trackLifetime  42378 non-null  int64  
 4   xCenter        42378 non-null  float64
 5   yCenter        42378 non-null  float64
 6   heading        42378 non-null  float64
 7   width          42378 non-null  float64
 8   length         42378 non-null  float64
 9   xVelocity      42378 non-null  float64
 10  yVelocity      42378 non-null  float64
 11  class          42378 non-null  object 
 12  pairs          42378 non-null  object 
 13  num_of_pairs   42378 non-null  int64  
 14  hx             42378 non-null  float64
 15  hy             42378 non-null  float64
dtypes: float64(9), int64(5), object(2)
memory usage: 5.2+ MB

### Create pairs of df's with changed columns names

In [6]:
def intialize_empty_df_ttc_format(letter_index_suffix):
    empty_df_ttc_format = pd.DataFrame(columns=['x' +        letter_index_suffix, 
                               'y' +        letter_index_suffix, 
                               'vx' +       letter_index_suffix, 
                               'vy' +       letter_index_suffix, 
                               'hx' +       letter_index_suffix,
                               'hy' +       letter_index_suffix,
                               'length' +   letter_index_suffix, 
                               'width' +    letter_index_suffix])
    return empty_df_ttc_format
# display(intialize_empty_df_ttc_format('_j'))
# display(intialize_empty_df_ttc_format('_i'))

In [7]:
start_time=time.time()

def calculate_ttc_and_add_to_pairs_column(df_temp):
    for i in tqdm(range(len(df_temp)), desc='Proccessing pairs for TTC'):
        # if there are pairs with the pedestrian, get its trajectory and the iterate peers
        if df_temp.loc[i, 'num_of_pairs'] >= 1:
            # intialize an empty df for pedestrian (object i) and insert pedestrian values  
            df_i = intialize_empty_df_ttc_format('_i')
            values_i = df_temp.loc[i, ['xCenter', 'yCenter','xVelocity','yVelocity',
                                    'hx', 'hy', 'length', 'width']].values
            df_i= pd.concat([df_i, pd.DataFrame([values_i], columns=df_i.columns)], ignore_index=True)
            # get pairs info from the pairs column
            object_j_pairs_list =  ast.literal_eval(df_temp.loc[i, 'pairs'])
            # iterate over peers j_n of pedestrian i
            j_n=1 # peer_number
            
            for index, (peer_row, peer_object_j_class) in enumerate(object_j_pairs_list):
                # initialize empty df_j for peer j_n and insert peer values
                df_j_n = intialize_empty_df_ttc_format('_j')
                values_j_n = df_temp.loc[peer_row, ['xCenter', 'yCenter','xVelocity','yVelocity',
                                                'hx', 'hy', 'length', 'width']].values
                df_j_n= pd.concat([df_j_n, pd.DataFrame([values_j_n], columns= df_j_n.columns)], ignore_index=True)
                # calculate TTC between pedestrian i and peer j_n
                ttc_i_j_n=TTC(  pd.concat([df_i,df_j_n], axis=1)   ,'dataframe')['TTC'][0]
                # add the ttc value to the list object_j_pairs_list
                object_j_pairs_list[index] = [peer_row, peer_object_j_class, ttc_i_j_n]
                j_n+=1
                
            # add the new object_j_pairs_list to the df_temp
            df_temp.at[i, 'pairs'] = str(object_j_pairs_list)
    
    return df_temp

dfs=load_dfs(file_name_data_with_pairs_list, root_dir, folder_name)
dfs_with_ttc = []
for x in range(len(dfs)):
    print('df name:',file_name_data_with_pairs_list[x])
    df_temp = dfs[x]
    dfs_with_ttc.append(calculate_ttc_and_add_to_pairs_column(df_temp))

total_time= time.time() - start_time
print('total_time for creating dfs_with_ttc:',total_time/60, 'minutes')

df name: 18_data_with_heading_vectors.csv


Proccessing pairs for TTC: 100%|██████████| 42378/42378 [09:40<00:00, 73.06it/s]  


df name: 19_data_with_heading_vectors.csv


Proccessing pairs for TTC: 100%|██████████| 33777/33777 [04:49<00:00, 116.80it/s] 


df name: 20_data_with_heading_vectors.csv


...

df name: 28_data_with_heading_vectors.csv


Proccessing pairs for TTC: 100%|██████████| 56613/56613 [08:18<00:00, 113.61it/s] 


df name: 29_data_with_heading_vectors.csv


Proccessing pairs for TTC: 100%|██████████| 67337/67337 [12:54<00:00, 86.92it/s]  

total_time for creating dfs_with_ttc: 120.68855760892232 minutes


In [7]:
# df_temp.to_csv('26_with_ttc_to_delete.csv', index=False)

In [8]:
recordingId_list = file_name_data_with_pairs_list 
for i in range(len(recordingId_list)):
    recordingId_list[i]=recordingId_list[i].replace('_data_with_heading_vectors.csv', '')
recordingId_list

['18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29']

In [9]:
### save to csv files  ###
os.chdir(root_dir)

def save_dfs(dfs_with_ttc, recordingId_list):
    # create folder to save files 
    folder_name='07_data_with_ttc'
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    # change to working dir
    os.chdir(folder_name)
    
    # initialize a list of the files to return by the function
    file_name_data_with_ttc_list=[]
    
    # iterate dfs_with_ttc and save
    for x in tqdm( range(len(dfs_with_ttc)) , desc="Saving CSV files" ):
        # save as CSV file
        file_name=str(recordingId_list[x]) + '_data_with_ttc' + '.csv'
        dfs[x].to_csv(file_name, index=False)
        # append the file name to the list
        file_name_data_with_ttc_list.append(file_name)
        
    print(f"files saved to: {os.getcwd()}")
    # change back to parent directory
    os.chdir('..')

    return file_name_data_with_ttc_list

file_name_data_with_ttc_list = save_dfs(dfs_with_ttc, recordingId_list)

Saving CSV files: 100%|██████████| 12/12 [00:04<00:00,  2.85it/s]

files saved to: C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\07_data_with_ttc


Now we have a csv files that contains ttc value for each pair!